In [35]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pandas as pd

def l2_to_cosine_similarity(l2):
    return 1 / (1 + l2)

def search_model(text_seach):
    # โหลดโมเดลที่เร็วขึ้น
    model = SentenceTransformer("ml/model")

    # โหลด FAISS index & sentences
    sentences = np.load("encode/sentences.npy")
    index = faiss.read_index("encode/faiss_index.bin")

    # Create Vector Search Index
    query = model.encode([text_seach])
    query = np.array(query).astype('float32')

    # ค้นหา Top 1 คล้ายที่สุด
    k=1
    distances, indices = index.search(query, k)     # ค้นหา

    threshold = 0.9
    best_match = None
    best_similarity = 0

    for i, dist in zip(indices[0], distances[0]):
        similarity = l2_to_cosine_similarity(dist)
        if similarity > threshold and similarity > best_similarity:
            best_match = sentences[i]
            best_similarity = similarity

    # print("Best Match:", best_match)
    return best_match, best_similarity

def search_contain(text_seach, df):
    match_sn = [sn for sn in df['SerialNumber'] if text_seach in sn]
    return match_sn

def search_result(best_match, best_similarity, df, text_seach):
    if best_match:
        matched_data = df[df["SerialNumber"] == best_match].to_dict(orient="records")
        res = {
            "best_similarity": best_similarity,
            "detail": matched_data,
            "process": "model"
        }
        print(res)
    else:
        matched_sn = search_contain(text_seach, df)

        if matched_sn:
            matched_data = df[df["SerialNumber"].isin(matched_sn)].to_dict(orient="records")

            if matched_data:
                res = {
                    "best_similarity": best_similarity,
                    "detail": matched_data,
                    "process": "contain"
                }
                print(res)
            else:
                print("serial number not found in contain search")
        else:
            print("serial number not found in model search")
    

In [36]:
text_seach = "CY26"
best_match, best_similarity = search_model(text_seach)
dataset = pd.read_csv("encode/data.csv")
search_result(best_match, best_similarity, dataset, text_seach)

{'best_similarity': 0, 'detail': [{'SerialNumber': 'CY26462438', 'Customer': 'Jane Smith', 'Product': 'Smartwatch', 'StartDate': '2023-10-17', 'EndDate': '2024-05-14'}], 'process': 'contain'}


In [37]:
import numpy as np
print(np.__version__)


1.26.4


In [1]:
pip install numpy==2.1.3


   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.9 MB 4.2 MB/s eta 0:00:03
   ---- ----------------------------------- 1.6/12.9 MB 4.0 MB/s eta 0:00:03
   ------- -------------------------------- 2.4/12.9 MB 3.7 MB/s eta 0:00:03
   --------- ------------------------------ 3.1/12.9 MB 3.8 MB/s eta 0:00:03
   ------------ --------------------------- 3.9/12.9 MB 3.9 MB/s eta 0:00:03
   --------------- ------------------------ 5.0/12.9 MB 3.9 MB/s eta 0:00:03
   ----------------- ---------------------- 5.8/12.9 MB 3.9 MB/s eta 0:00:02
   -------------------- ------------------- 6.6/12.9 MB 3.9 MB/s eta 0:00:02
   ----------------------- ---------------- 7.6/12.9 MB 4.0 MB/s eta 0:00:02
   -------------------------- ------------- 8.4/12.9 MB 4.0 MB/s eta 0:00:02
   ---------------------------- ----------- 9.2/12.9 MB 4.0 MB/s eta 0:00:01
   ------------------------------ --------- 10.0/12.9 MB 4.0 MB/s eta 0:00:01
   --

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-learn 1.4.0 requires numpy<2.0,>=1.19.5, but you have numpy 2.1.3 which is incompatible.
scipy 1.10.1 requires numpy<1.27.0,>=1.19.5, but you have numpy 2.1.3 which is incompatible.
streamlit 1.28.1 requires numpy<2,>=1.19.3, but you have numpy 2.1.3 which is incompatible.
